In [18]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.dtr ,2.xgbr, 3.ann)

In [20]:
from xgboost import XGBRegressor
new_model=XGBRegressor()
new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in1110.h5')

#pickle for dtr
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
  #  new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_modelrin3outr6.h5')
# Show the model architecture
#new_model.summary()

In [21]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new 1110 for colab.csv')
Xtest=test.iloc[:,:5].values
Ytest = test.iloc[:,5:].values
print(Xtest[0])


[2.00000000e+00 5.04800000e+01 3.55247982e-01 3.00000000e+00
 1.00000000e+03]


In [22]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [23]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [24]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [25]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [26]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(29,49,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,NPCweight,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [27]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

[[1.00e+00 5.34e+01 3.00e-01 7.50e-01 1.00e+02]
 [1.00e+00 5.33e+01 3.60e-01 7.50e-01 5.00e+02]
 [1.00e+00 5.89e+01 3.90e-01 3.00e+00 2.00e+02]
 [1.00e+00 5.20e+01 4.50e-01 7.50e-01 7.00e+02]
 [2.00e+00 6.04e+01 3.40e-01 1.00e+00 8.00e+02]
 [1.00e+00 5.42e+01 3.20e-01 7.50e-01 2.00e+02]
 [1.00e+00 5.78e+01 4.70e-01 7.50e-01 3.00e+02]
 [1.00e+00 5.04e+01 4.40e-01 7.50e-01 6.00e+02]
 [1.00e+00 5.82e+01 3.50e-01 7.50e-01 1.00e+02]
 [2.00e+00 5.59e+01 3.00e-01 3.00e+00 1.00e+03]
 [1.00e+00 5.71e+01 4.40e-01 1.00e+00 1.00e+03]
 [2.00e+00 5.75e+01 3.50e-01 7.50e-01 1.00e+03]
 [1.00e+00 5.80e+01 4.10e-01 3.00e+00 7.00e+02]
 [2.00e+00 5.77e+01 3.20e-01 3.00e+00 5.00e+02]
 [2.00e+00 5.19e+01 4.80e-01 3.00e+00 3.00e+02]
 [2.00e+00 5.77e+01 3.10e-01 7.50e-01 2.00e+02]
 [2.00e+00 6.07e+01 3.80e-01 1.00e+00 2.00e+02]
 [1.00e+00 5.97e+01 4.20e-01 7.50e-01 5.00e+02]
 [1.00e+00 6.08e+01 3.30e-01 1.00e+00 3.00e+02]
 [1.00e+00 5.79e+01 4.70e-01 7.50e-01 8.00e+02]]
[12.065145 24.85212  24.137468 36.79464

In [28]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()]
    parents_pool=mixsort[:,0:5]
    parents_results=mixsort[:,5]
    return parents_pool


In [29]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
  
    return crossover_pool






In [30]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.9,1.1)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#X=mating_pool[:,0]
#mutation_pool=mating_pool[:,1:]*mutation_rate
#yui=np.column_stack((X,mutation_pool))
#print(mutation_pool,yui)

In [31]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 5].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:5]
    return new_mating_pool


test2=survivor_selection()
print(test2)

[[2.00000000e+00 6.04000000e+01 3.40000000e-01 1.00000000e+00
  8.00000000e+02]
 [1.00000000e+00 5.20000000e+01 4.50000000e-01 7.50000000e-01
  7.00000000e+02]
 [1.00000000e+00 5.04000000e+01 4.40000000e-01 7.50000000e-01
  6.00000000e+02]
 [1.00000000e+00 5.83182850e+01 3.12978273e-01 3.12978273e+00
  1.04326091e+03]
 [1.00000000e+00 5.97000000e+01 4.20000000e-01 7.50000000e-01
  5.00000000e+02]
 [1.00000000e+00 5.71000000e+01 4.40000000e-01 1.00000000e+00
  1.00000000e+03]
 [1.00000000e+00 6.08000000e+01 3.30000000e-01 1.00000000e+00
  3.00000000e+02]
 [1.00000000e+00 5.80000000e+01 4.10000000e-01 3.00000000e+00
  7.00000000e+02]
 [1.00000000e+00 5.79000000e+01 4.70000000e-01 7.50000000e-01
  8.00000000e+02]
 [2.00000000e+00 5.75000000e+01 3.50000000e-01 7.50000000e-01
  1.00000000e+03]
 [2.00000000e+00 5.19000000e+01 4.80000000e-01 3.00000000e+00
  3.00000000e+02]
 [2.00000000e+00 6.33259373e+01 3.96439146e-01 1.04326091e+00
  2.08652182e+02]
 [1.00000000e+00 5.42000000e+01 3.200000

In [32]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


[[2.00000000e+00 6.04000000e+01 3.40000000e-01 1.00000000e+00
  8.00000000e+02]
 [1.00000000e+00 5.20000000e+01 4.50000000e-01 7.50000000e-01
  7.00000000e+02]
 [1.00000000e+00 6.02190198e+01 3.23178997e-01 8.07947492e-01
  1.07726332e+03]
 [1.00000000e+00 5.04000000e+01 4.40000000e-01 7.50000000e-01
  6.00000000e+02]
 [1.00000000e+00 5.97000000e+01 4.20000000e-01 7.50000000e-01
  5.00000000e+02]
 [1.00000000e+00 5.71000000e+01 4.40000000e-01 1.00000000e+00
  1.00000000e+03]
 [1.00000000e+00 6.08000000e+01 3.30000000e-01 1.00000000e+00
  3.00000000e+02]
 [1.00000000e+00 5.80000000e+01 4.10000000e-01 3.00000000e+00
  7.00000000e+02]
 [1.00000000e+00 5.79000000e+01 4.70000000e-01 7.50000000e-01
  8.00000000e+02]
 [2.00000000e+00 5.75000000e+01 3.50000000e-01 7.50000000e-01
  1.00000000e+03]
 [2.00000000e+00 5.19000000e+01 4.80000000e-01 3.00000000e+00
  3.00000000e+02]
 [2.00000000e+00 6.53898837e+01 4.09360063e-01 3.23178997e+00
  2.15452665e+02]
 [1.00000000e+00 5.42000000e+01 3.200000

In [33]:
initial_gen=0
max_gen=100000

import numpy as np


while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=65 : break
   #elif np.sum(test[0])>=97.5: break
   

Streaming output truncated to the last 5000 lines.
 [2.00000000e+00 5.61747511e+01 3.27267227e-01 7.32714145e-01
  6.21135911e+02]
 [2.00000000e+00 5.71742994e+01 3.07175612e-01 7.43166804e-01
  6.06916327e+02]
 [2.00000000e+00 5.72875448e+01 3.40406373e-01 9.49978812e-01
  8.00956172e+02]
 [1.00000000e+00 5.54547919e+01 3.57157681e-01 1.06075938e+00
  7.17393377e+02]
 [2.00000000e+00 4.39994155e+01 3.48565831e-01 7.88761386e-01
  6.70125523e+02]
 [2.00000000e+00 5.51399868e+01 3.49229889e-01 7.90264067e-01
  6.80403741e+02]
 [2.00000000e+00 4.17425375e+01 4.08033843e-01 7.00128348e-01
  4.52393872e+02]
 [2.00000000e+00 7.02881890e+01 3.31528361e-01 9.77124709e-01
  6.53932701e+02]
 [1.00000000e+00 4.46499197e+01 4.13493845e-01 7.33499384e-01
  6.67430141e+02]
 [2.00000000e+00 4.96768029e+01 3.99691304e-01 6.14927158e-01
  4.74634859e+02]
 [2.00000000e+00 4.95550573e+01 3.95202999e-01 6.11468905e-01
  4.73873425e+02]
 [2.00000000e+00 4.96226411e+01 3.95741982e-01 6.06659822e-01
  4.745

In [34]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))


[15.789658   15.185109   46.573788   -0.08453351 15.103639  ] [2.00000000e+00 5.65651520e+01 3.96712861e-01 6.58891249e-01
 4.36545712e+02]
[[15.789658   15.185109   46.573788   -0.08453351 15.103639  ]
 [16.887768   15.590703   46.477047    0.04923642 16.787579  ]
 [24.172886   20.497715   46.23534     2.668419   16.940355  ]
 [16.969715   15.402511   45.461987    0.33128646 16.412945  ]
 [15.574674   15.018666   45.07758     0.40274262 17.148413  ]
 [27.01694     9.4625225  42.339592    2.458698   21.420483  ]
 [16.132801   14.551862   42.085255    0.48586175 17.402592  ]
 [13.553871   16.135826   41.991703    0.95696396 19.260489  ]
 [15.061726   16.783798   41.85382     0.9828476  19.331242  ]
 [17.295835   10.091716   41.735977    2.9367855  21.642973  ]
 [12.567162   26.890862   41.135925    0.15071149 14.563636  ]
 [27.006756   20.389057   41.050182    0.6307574  14.683412  ]
 [21.229866   25.606144   40.867897    0.5056268  14.483314  ]
 [16.420233   23.780796   40.413864    0.